# Arch and Freq Invariance

This notebook shows the architecture and cpu frequency invariance features of EAS.

In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)
# Comment the follwing line to disable devlib debugging statements
# logging.getLogger('ssh').setLevel(logging.DEBUG)

In [2]:
# Generate plots inline
%pylab inline

import json
import os

import subprocess

# Support to access the remote target
import devlib
from env import TestEnv

# Support for trace events analysis
from trace import Trace
#from trace_analysis import TraceAnalysis

# Support to configure and run RTApp based workloads
from wlgen import RTA

# Support for performance analysis of RTApp workloads
from perf_analysis import PerfAnalysis

# Suport for FTrace events parsing and visualization
import trappy

Populating the interactive namespace from numpy and matplotlib


# Test environment setup

In [3]:
# Setup a target configuration
androidsdk_path=os.path.join(os.getcwd(), "../../../android-sdk")
my_target_conf = {
    
    "platform"    : 'android',
    "board"       : "hikey960",
    "device" : "0123456789ABCDEF",
    
    "ANDROID_HOME" : androidsdk_path,
    "rtapp-calib" : {"0": 302, "1": 302, "2": 304, "3": 304, "4": 136, "5": 137, "6": 136, "7": 136},

     "emeter" : {
        "instrument" : "acme",
        "conf" : {
            # Absolute path to the iio-capture binary on the host
            'iio-capture' : '/usr/bin/iio-capture',
            # Default host name of the BeagleBone Black
            'ip_address'     : '10.169.36.36',
        },
        "channel_map" : {
            "Device0" : 0, # iio:device0
            "Device1" : 1, # iio:device0
        }
    },
    
    # Define devlib module to load
    #"modules"     : [
    #    'bl',           # enable big.LITTLE support
    #    'cpufreq'       # enable CPUFreq support
    #],

    # Account to access the remote target
    #"host"        : '192.168.1.101',
    "host"        : '10.169.36.74',
    "username"    : 'root',
    "password"    : '',

    # Comment the following line to force rt-app calibration on your target
#     "rtapp-calib" : {
#         '0': 361, '1': 138, '2': 138, '3': 352, '4': 360, '5': 353
#     }
}

# Setup the required Test Environment supports
my_tests_conf = {
    
    # Binary tools required to run this experiment
    # These tools must be present in the tools/ folder for the architecture
    "tools"   : ['rt-app', 'taskset', 'trace-cmd'],
    
    # FTrace events end buffer configuration
    "ftrace"  : {
         "events" : [
            "cpu_frequency",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_switch",
            'dequeue_task_fair', 
            'enqueue_task_fair', 
            'set_next_entity',

                    
            "cpu_capacity",
            
            #/sys/kernel/debug/tracing/events/sched
            "sched_blocked_reason",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_contrib_scale_f",
            "sched_cpu_hotplug",
            "sched_energy_diff",
            "sched_kthread_stop",
            "sched_kthread_stop_ret",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_migrate_task",
            "sched_move_numa",
            "sched_pi_setprio",
            "sched_process_exec",
            "sched_process_exit",
            "sched_process_fork",
            "sched_process_free",
            "sched_process_wait",
            "sched_stat_blocked",
            "sched_stat_iowait",
            "sched_stat_runtime",
            "sched_stat_sleep",
            "sched_stat_wait",
            "sched_stick_numa",
            "sched_swap_numa",
            "sched_switch",
            "sched_tune_boostgroup_update",
            "sched_tune_config",
            "sched_tune_tasks_update",
            "sched_tune_filter",
            "sched_wait_task",
            "sched_wake_idle_without_ipi",
            "sched_wakeup",
            "sched_wakeup_new",
         ],
         "buffsize" : 10240
    },

    "results_dir" : "arch_and_freq_invariance",
}

In [4]:
# Support to access the remote target
import devlib
from env import TestEnv

# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration   (my_test_conf)
te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)
target = te.target

11:37:04  INFO    : Using base path: /home/steven/lisa/lisa-github
11:37:04  INFO    : Loading custom (inline) target configuration
11:37:04  INFO    : Loading custom (inline) test configuration
11:37:04  WARNING : Wipe previous contents of the results folder:
11:37:04  WARNING :    /home/steven/lisa/lisa-github/results/arch_and_freq_invariance
11:37:04  INFO    : External tools using:
11:37:04  INFO    :    ANDROID_HOME: /home/steven/lisa/lisa-github/ipynb/workshop/../../../android-sdk
11:37:04  INFO    :    CATAPULT_HOME: /home/steven/lisa/lisa-github/tools/catapult
11:37:04  INFO    : Devlib modules to load: ['bl', 'cpuidle', 'cpufreq']
11:37:04  INFO    : Connecting Android target [0123456789ABCDEF]
11:37:04  INFO    : Connection settings:
11:37:04  INFO    :    {'device': '0123456789ABCDEF'}
11:37:05  INFO    : Initializing target workdir:
11:37:05  INFO    :    /data/local/tmp/devlib-target
11:37:07  INFO    : Attempting to read energy model from target
11:37:08  INFO    : Topolo

# Workload configuration

In [5]:
# Support to configure and run RTApp based workloads
from wlgen import RTA, Periodic, Ramp

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp = RTA(target, 'simple', calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(
    # 1. generate a "profile based" set of tasks
    kind='profile',
    
    # 2. define the "profile" of each task
    params={
        
        
        'task_p50': Periodic(
            period_ms=400,         # period
            duty_cycle_pct=50,     # duty cycle
            duration_s=10,         # duration    
            cpus=str(target.bl.bigs[0])   # pinned on first big CPU
        ).get(),
        
        'task_r20': Ramp(
            start_pct=5,           # intial load
            end_pct=95,            # end load
            delta_pct=10,          # load % increase...
            time_s=1,              # ... every 1[s]            
        ).get(),
    },
    
    # 4. use this folder for task logfiles
    run_dir=target.working_directory
    
);

11:37:20  INFO    : Setup new workload simple
11:37:20  INFO    : Workload duration defined by longest task
11:37:20  INFO    : Default policy: SCHED_OTHER
11:37:21  INFO    : ------------------------
11:37:21  INFO    : task [task_p50], sched: using default policy
11:37:21  INFO    :  | loops count: 1
11:37:21  INFO    :  | CPUs affinity: 4
11:37:21  INFO    : + phase_000001: duration 10.000000 [s] (25 loops)
11:37:21  INFO    : |  period   400000 [us], duty_cycle  50 %
11:37:21  INFO    : |  run_time 200000 [us], sleep_time 200000 [us]
11:37:21  INFO    : ------------------------
11:37:21  INFO    : task [task_r20], sched: using default policy
11:37:21  INFO    :  | loops count: 1
11:37:21  INFO    : + phase_000001: duration 1.000000 [s] (10 loops)
11:37:21  INFO    : |  period   100000 [us], duty_cycle   5 %
11:37:21  INFO    : |  run_time   5000 [us], sleep_time  95000 [us]
11:37:21  INFO    : + phase_000002: duration 1.000000 [s] (10 loops)
11:37:21  INFO    : |  period   100000 [

# Workload execution round 1

In [6]:
# Set performance governor
#
# For Juno big/LITTLE CPUs
#
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

target.cpufreq.set_all_governors('performance')

logging.info("Target current governor: %s",
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor")
            )
logging.info("Target LITTLE CPU max CPUfreq:%s\n\t\t    Target LITTLE CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq")
             )

logging.info("Target big CPU max CPUfreq:%s \n\t\t    Target big CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_cur_freq")
             )

11:37:25  INFO    : Target ABI: arm64, CPus: ['A53', 'A53', 'A53', 'A53', 'A73', 'A73', 'A73', 'A73']
11:37:25  INFO    : Target current governor: performance
11:37:25  INFO    : Target LITTLE CPU max CPUfreq:1844000
		    Target LITTLE CPU current CPUfreq: 1844000
11:37:25  INFO    : Target big CPU max CPUfreq:2362000 
		    Target big CPU current CPUfreq: 2362000


In [7]:
logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(out_dir=te.res_dir, cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
nrg_report = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

11:37:31  INFO    : #### Setup FTrace
11:37:34  INFO    : #### Start energy sampling
11:37:35  INFO    : #### Start RTApp execution
11:37:35  INFO    : Workload execution START:
11:37:35  INFO    :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/simple_00.json 2>&1
11:37:46  INFO    : #### Read energy consumption: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/energy.json
11:37:46  ERROR   : /usr/bin/iio-capture terminated for Device1 (iio:device1)
11:37:46  ERROR   : [Unable to create IIO context
]
11:37:46  ERROR   : /usr/bin/iio-capture terminated for Device0 (iio:device0)
11:37:46  ERROR   : [Unable to create IIO context
]
11:37:46  INFO    : #### Stop FTrace
11:37:46  INFO    : #### Save FTrace: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/trace.dat
11:37:48  INFO    : #### Save platform description: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/platform.json


# Collected results

In [8]:
# All data are produced in the output folder defined by the TestEnv module
logging.info('Content of the output folder %s', te.res_dir)
!ls -la {te.res_dir}

11:37:59  INFO    : Content of the output folder /home/steven/lisa/lisa-github/results/arch_and_freq_invariance


total 7592
drwxrwxr-x 1 steven steven     326 12月 25 11:37 .
drwxrwxr-x 1 steven steven    1552 12月 25 11:37 ..
-rw-rw-r-- 1 steven steven       2 12月 25 11:37 energy.json
-rw-rw-r-- 1 steven steven       2 12月 25 11:37 energy_stats.json
-rw-rw-r-- 1 steven steven     307 12月 25 11:37 output.log
-rw-rw-r-- 1 steven steven    1526 12月 25 11:37 platform.json
-rw-r--r-- 1 steven steven    3260 12月 25 11:37 rt-app-task_p50-0.log
-rw-r--r-- 1 steven steven   24960 12月 25 11:37 rt-app-task_r20-0.log
-rw-r--r-- 1 steven steven   12560 12月 25 11:37 rt-app-task_r20-1.log
-rw-r--r-- 1 steven steven    5120 12月 25 11:37 rt-app-task_r20_5-60-1.log
-rw-r--r-- 1 steven steven    3265 12月 25 11:37 simple_00.json
-rw-r--r-- 1 steven steven 7696384 12月 25 11:37 trace.dat


In [9]:
# Inspect the JSON file used to run the application
with open('{}/simple_00.json'.format(te.res_dir), 'r') as fh:
    rtapp_json = json.load(fh, )
logging.info('Generated RTApp JSON file:')
#print json.dumps(rtapp_json, indent=4, sort_keys=True)

11:38:00  INFO    : Generated RTApp JSON file:


In [10]:
# Dump the energy measured for the LITTLE and big clusters
logging.info('Energy: %s', nrg_report.report_file)
print json.dumps(nrg_report.channels, indent=4, sort_keys=True)

11:38:00  INFO    : Energy: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/energy.json


{}


In [11]:
# Dump the platform descriptor, which could be useful for further analysis
# of the generated results
logging.info('Platform description: %s', plt_file)
#print json.dumps(plt, indent=4, sort_keys=True)

11:38:01  INFO    : Platform description: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/platform.json


# Trace inspection

In [12]:
# Suport for FTrace events parsing and visualization
import trappy

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

In [13]:
events_to_parse = my_tests_conf['ftrace']['events']

trace = Trace(plt, te.res_dir, events_to_parse)

ftrace = trace.ftrace

trappy.plotter.plot_trace(ftrace, execnames=['task_p50', "task_r20" ])

/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.set_next_entity object at 0x7fe632b89450> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.cpu_frequency_devlib object at 0x7fe632b895d0> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.dequeue_task_fair object at 0x7fe632b897d0> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_wait_task object at 0x7fe632b898d0> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_stat_blo

# Workload Execution Round 2

In [14]:
# Set userspace governor with lowest CPUfreq
#
# For Juno big/LITTLE CPUs
#
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

target.cpufreq.set_all_governors('userspace')

LITTLE_min_freq=target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_min_freq")
big_min_freq=target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_min_freq")


target.write_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_setspeed", LITTLE_min_freq)
target.write_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_setspeed", big_min_freq)

logging.info("Target current governor: %s",
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor")
            )
logging.info("Target big CPU min CPUfreq:%s\n\t\t    Target big CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_min_freq"),
             target.read_value("/sys/devices/system/cpu/cpu4/cpufreq/scaling_cur_freq")
             )

logging.info("Target LITTLE CPU min CPUfreq:%s \n\t\t    Target LITTLE CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_min_freq"),
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq")
             )

11:38:29  INFO    : Target ABI: arm64, CPus: ['A53', 'A53', 'A53', 'A53', 'A73', 'A73', 'A73', 'A73']
11:38:29  INFO    : Target current governor: userspace
11:38:30  INFO    : Target big CPU min CPUfreq:903000
		    Target big CPU current CPUfreq: 903000
11:38:30  INFO    : Target LITTLE CPU min CPUfreq:533000 
		    Target LITTLE CPU current CPUfreq: 533000


In [15]:
logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(out_dir=te.res_dir, cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
nrg_report = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

11:38:31  INFO    : #### Setup FTrace
11:38:35  INFO    : #### Start energy sampling
11:38:36  INFO    : #### Start RTApp execution
11:38:36  INFO    : Workload execution START:
11:38:36  INFO    :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/simple_00.json 2>&1
11:39:16  INFO    : #### Read energy consumption: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/energy.json
11:39:16  ERROR   : /usr/bin/iio-capture terminated for Device1 (iio:device1)
11:39:16  ERROR   : [Unable to create IIO context
]
11:39:16  ERROR   : /usr/bin/iio-capture terminated for Device0 (iio:device0)
11:39:16  ERROR   : [Unable to create IIO context
]
11:39:16  INFO    : #### Stop FTrace
11:39:17  INFO    : #### Save FTrace: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/trace.dat
11:39:21  INFO    : #### Save platform description: /home/steven/lisa/lisa-github/results/arch_and_freq_invariance/platform.json


In [16]:
# All data are produced in the output folder defined by the TestEnv module
logging.info('Content of the output folder %s', te.res_dir)
!ls -la {te.res_dir}

11:39:21  INFO    : Content of the output folder /home/steven/lisa/lisa-github/results/arch_and_freq_invariance


total 13960
drwxrwxr-x 1 steven steven      358 12月 25 11:39 .
drwxrwxr-x 1 steven steven     1552 12月 25 11:37 ..
-rw-rw-r-- 1 steven steven        2 12月 25 11:39 energy.json
-rw-rw-r-- 1 steven steven        2 12月 25 11:39 energy_stats.json
-rw-rw-r-- 1 steven steven      307 12月 25 11:39 output.log
-rw-rw-r-- 1 steven steven     1526 12月 25 11:39 platform.json
-rw-r--r-- 1 steven steven     3260 12月 25 11:39 rt-app-task_p50-0.log
-rw-r--r-- 1 steven steven    24960 12月 25 11:39 rt-app-task_r20-0.log
-rw-r--r-- 1 steven steven    12560 12月 25 11:39 rt-app-task_r20-1.log
-rw-r--r-- 1 steven steven     5120 12月 25 11:39 rt-app-task_r20_5-60-1.log
-rw-r--r-- 1 steven steven     3265 12月 25 11:39 simple_00.json
-rw-r--r-- 1 steven steven 14217216 12月 25 11:39 trace.dat
drwxrwxr-x 1 steven steven     2664 12月 25 11:38 .trace.dat.cache


# Collected Results

# Trace inspection

In [17]:
# Suport for FTrace events parsing and visualization
import trappy

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:201: UserWarning: Cached data is from another trace, invalidating cache.
  warnings.warn(warnstr)


In [18]:
events_to_parse = my_tests_conf['ftrace']['events']

trace = Trace(plt, te.res_dir, events_to_parse)

ftrace = trace.ftrace

trappy.plotter.plot_trace(ftrace, execnames=['task_p50', "task_r20" ])

/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.set_next_entity object at 0x7fe6330235d0> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.cpu_frequency_devlib object at 0x7fe633023710> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.dequeue_task_fair object at 0x7fe633023910> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_wait_task object at 0x7fe633023a10> from cache, reading it from trace
  warnings.warn(warnstr)
/home/steven/lisa/lisa-github/libs/trappy/trappy/ftrace.py:274: UserWarning: TRAPpy: Couldn't read <trappy.dynamic.sched_stat_blo

In [19]:
target.cpufreq.set_all_governors('schedutil')

''

In [20]:
target.cpufreq.get_all_governors()

{'0': 'schedutil',
 '1': 'schedutil',
 '2': 'schedutil',
 '3': 'schedutil',
 '4': 'schedutil',
 '5': 'schedutil',
 '6': 'schedutil',
 '7': 'schedutil'}